In [161]:
import pandas as pd
from pymongo import MongoClient
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import time
import json

In [2]:
client_id = os.getenv("four")
client_secret = os.getenv("foursec")

In [13]:
local = pd.read_csv("../data/coor.csv",index_col="Unnamed: 0")
local

,latitude,longitude
Austin,30.268735,-97.745209
Hamburg,53.597305,9.976252
Newcastle,54.971346,-1.611141


In [4]:
# Austin = {'type': 'Point', 'coordinates': local[local["coordinates"]]}
# Hamburg = {'type': 'Point', 'coordinates': [ham_lat, ham_lon]}
# Newcastle = {'type': 'Point', 'coordinates': [ncas_lat, ncas_lon]}

In [5]:
categories = pd.read_csv("../data/categories.csv")

In [6]:
categories.sample(3)

,category,categoryId
659,Corporate Coffee Shop,5665c7b9498e7d8a4f2c0f06
104,Event,4d4b7105d754a06373d81259
678,Middle School,4f4533814b9074f6e4fb0106


In [7]:
design = categories.categoryId[categories["category"] == "Design Studio"].values[0]
#schools
Nursery = categories.categoryId[categories["category"] == "Nursery School"].values[0]#Nursery School
Preschool = categories.categoryId[categories["category"] == "Preschool"].values[0]
Elementary = categories.categoryId[categories["category"] == "Elementary School"].values[0]

In [8]:
design,Nursery,Preschool,Elementary

('4bf58dd8d48988d1f4941735',
 '4f4533814b9074f6e4fb0107',
 '52e81612bcbc57f1066b7a45',
 '4f4533804b9074f6e4fb0105')

In [9]:
lista = ["Tech Startup","Airport Terminal","Train Station"]

In [10]:
cat_dic = []
for l in lista:
    dicc = {l:categories.categoryId[categories["category"] == l].values[0] }
    cat_dic.append(dicc)
cat_dic

[{'Tech Startup': '4bf58dd8d48988d125941735'},
 {'Airport Terminal': '4bf58dd8d48988d1eb931735'},
 {'Train Station': '4bf58dd8d48988d129951735'}]

In [11]:
venues_list = ["Design Studio","Nursery School","Preschool","Elementary School","Tech Startup","Coffee Shop",
               "Airport Terminal","Train Station",
              "Nightlife Spot","Vegetarian / Vegan Restaurant","Basketball Stadium",
               "Veterinarian","Pet Service","Pet Store"]      

In [74]:
# cat_dic = []
# for l in venues_list:
#     dicc = {l:categories.categoryId[categories["category"] == l].values[0] }
#     cat_dic.append(dicc)
# cat_dic

cat_dic = {}
for l in venues_list:
    cat_dic[l] = categories.categoryId[categories["category"] == l].values[0]
cat_dic

{'Design Studio': '4bf58dd8d48988d1f4941735',
 'Nursery School': '4f4533814b9074f6e4fb0107',
 'Preschool': '52e81612bcbc57f1066b7a45',
 'Elementary School': '4f4533804b9074f6e4fb0105',
 'Tech Startup': '4bf58dd8d48988d125941735',
 'Coffee Shop': '4bf58dd8d48988d1e0931735',
 'Airport Terminal': '4bf58dd8d48988d1eb931735',
 'Train Station': '4bf58dd8d48988d129951735',
 'Nightlife Spot': '4d4b7105d754a06376d81259',
 'Vegetarian / Vegan Restaurant': '4bf58dd8d48988d1d3941735',
 'Basketball Stadium': '4bf58dd8d48988d18b941735',
 'Veterinarian': '4d954af4a243a5684765b473',
 'Pet Service': '5032897c91d4c4b30a586d69',
 'Pet Store': '4bf58dd8d48988d100951735'}

In [14]:
local

,latitude,longitude
Austin,30.268735,-97.745209
Hamburg,53.597305,9.976252
Newcastle,54.971346,-1.611141


In [70]:
Austin = {"Austin":[30.268735, -97.745209]}
Hamburg = {"Hamburg" : [53.5973055, 9.9762519]}
Newcastle = {"Newcastle": [54.971346, -1.611141]}
localizations = {"Austin":[30.268735, -97.745209],"Hamburg" : [53.5973055, 9.9762519],"Newcastle": [54.971346, -1.611141]}
localizations.keys()

dict_keys(['Austin', 'Hamburg', 'Newcastle'])

In [81]:
for loc,coor in localizations.items():
    for key,value in cat_dic.items():
        print(loc,coor[0],value)

Austin 30.268735 4bf58dd8d48988d1f4941735
Austin 30.268735 4f4533814b9074f6e4fb0107
Austin 30.268735 52e81612bcbc57f1066b7a45
Austin 30.268735 4f4533804b9074f6e4fb0105
Austin 30.268735 4bf58dd8d48988d125941735
Austin 30.268735 4bf58dd8d48988d1e0931735
Austin 30.268735 4bf58dd8d48988d1eb931735
Austin 30.268735 4bf58dd8d48988d129951735
Austin 30.268735 4d4b7105d754a06376d81259
Austin 30.268735 4bf58dd8d48988d1d3941735
Austin 30.268735 4bf58dd8d48988d18b941735
Austin 30.268735 4d954af4a243a5684765b473
Austin 30.268735 5032897c91d4c4b30a586d69
Austin 30.268735 4bf58dd8d48988d100951735
Hamburg 53.5973055 4bf58dd8d48988d1f4941735
Hamburg 53.5973055 4f4533814b9074f6e4fb0107
Hamburg 53.5973055 52e81612bcbc57f1066b7a45
Hamburg 53.5973055 4f4533804b9074f6e4fb0105
Hamburg 53.5973055 4bf58dd8d48988d125941735
Hamburg 53.5973055 4bf58dd8d48988d1e0931735
Hamburg 53.5973055 4bf58dd8d48988d1eb931735
Hamburg 53.5973055 4bf58dd8d48988d129951735
Hamburg 53.5973055 4d4b7105d754a06376d81259
Hamburg 53.59730

In [105]:
def place_store(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    Aus_venues =[]
    Ham_venues = []
    Ncas_venues = []
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            if loc == "Austin":
                Aus_venues.append(requests.get(url_query, params = parametros).json()["response"]["venues"])
                print(f"{loc},{cat} hecho")
            elif loc == "Hamburg":
                Ham_venues.append(requests.get(url_query, params = parametros).json()["response"]["venues"])
                print(f"{loc},{cat} hecho")
            elif loc == "Newcastle":
                Ncas_venues.append(requests.get(url_query, params = parametros).json()["response"]["venues"])
                print(f"{loc},{cat} hecho")             

    return [Aus_venues,Ham_venues,Ncas_venues]

    
    
total = place_store(localizations,cat_dic)

Austin,Design Studio hecho
Austin,Nursery School hecho
Austin,Preschool hecho
Austin,Elementary School hecho
Austin,Tech Startup hecho
Austin,Coffee Shop hecho
Austin,Airport Terminal hecho
Austin,Train Station hecho
Austin,Nightlife Spot hecho
Austin,Vegetarian / Vegan Restaurant hecho
Austin,Basketball Stadium hecho
Austin,Veterinarian hecho
Austin,Pet Service hecho
Austin,Pet Store hecho
Hamburg,Design Studio hecho
Hamburg,Nursery School hecho
Hamburg,Preschool hecho
Hamburg,Elementary School hecho
Hamburg,Tech Startup hecho
Hamburg,Coffee Shop hecho
Hamburg,Airport Terminal hecho
Hamburg,Train Station hecho
Hamburg,Nightlife Spot hecho
Hamburg,Vegetarian / Vegan Restaurant hecho
Hamburg,Basketball Stadium hecho
Hamburg,Veterinarian hecho
Hamburg,Pet Service hecho
Hamburg,Pet Store hecho
Newcastle,Design Studio hecho
Newcastle,Nursery School hecho
Newcastle,Preschool hecho
Newcastle,Elementary School hecho
Newcastle,Tech Startup hecho
Newcastle,Coffee Shop hecho
Newcastle,Airport Te

In [110]:
len(total)
len(total[2])

14

In [122]:
total_df = pd.DataFrame(total)

In [162]:
#with open('data.json', 'w') as file:
    #json.dump(data, file, indent=4)
    
with open('../data/total.json', 'w') as f:
    json.dump(total, f)
    

### a ver cómo hago dicc total sin lista de listas

In [128]:
def place_store(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues = []
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            
            venues.append(requests.get(url_query, params = parametros).json()["response"]["venues"])
            print(f"{loc},{cat} hecho")
           

    return venues

    
    
total_venues = place_store(localizations,cat_dic)

Austin,Design Studio hecho
Austin,Nursery School hecho
Austin,Preschool hecho
Austin,Elementary School hecho
Austin,Tech Startup hecho
Austin,Coffee Shop hecho
Austin,Airport Terminal hecho
Austin,Train Station hecho
Austin,Nightlife Spot hecho
Austin,Vegetarian / Vegan Restaurant hecho
Austin,Basketball Stadium hecho
Austin,Veterinarian hecho
Austin,Pet Service hecho
Austin,Pet Store hecho
Hamburg,Design Studio hecho
Hamburg,Nursery School hecho
Hamburg,Preschool hecho
Hamburg,Elementary School hecho
Hamburg,Tech Startup hecho
Hamburg,Coffee Shop hecho
Hamburg,Airport Terminal hecho
Hamburg,Train Station hecho
Hamburg,Nightlife Spot hecho
Hamburg,Vegetarian / Vegan Restaurant hecho
Hamburg,Basketball Stadium hecho
Hamburg,Veterinarian hecho
Hamburg,Pet Service hecho
Hamburg,Pet Store hecho
Newcastle,Design Studio hecho
Newcastle,Nursery School hecho
Newcastle,Preschool hecho
Newcastle,Elementary School hecho
Newcastle,Tech Startup hecho
Newcastle,Coffee Shop hecho
Newcastle,Airport Te

In [284]:
len(total_venues)

42

In [131]:
df_tot_venues_austin = pd.DataFrame(total_venues[0])
df_tot_venues_austin

,id,name,contact,location,categories,verified,stats,beenHere,venuePage,hereNow,referralId,venueChains,hasPerk
0,4bd339039854d13a2635fd4d,Kruger's Diamond Jewelers,{},"{'address': '722 congress Ave', 'crossStreet':...","[{'id': '4bf58dd8d48988d111951735', 'name': 'J...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",{'id': '169062418'},"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
1,53d2e4fe498ecb061bd1992e,Rae Cosmetics,{},"{'address': '237 W 2nd St', 'crossStreet': 'La...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",{'id': '427126994'},"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
2,512a4f36e4125c76886e6173,AT&T — Consumer Digital Experiences,{},"{'address': '909 Colorado St Fl 7', 'lat': 30....","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
3,4e06499d62e100e58168b6cb,Big Red Sun,{},"{'address': '1311 E Cesar Chavez St', 'crossSt...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
4,5a1c9082a5a3153c3bedf498,Bright Health,{},"{'address': '1135 W 6th St', 'lat': 30.2724780...","[{'id': '58daa1558bbb0b01f18ec1f1', 'name': 'I...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
5,5817a1a738faf203b56e5757,Awake° Studios,{},"{'address': '501 N Interstate 35 Ste 111', 'cr...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
6,49d926e6f964a5200a5e1fe3,Urbanspace Interiors,{},"{'address': '801 W 5th St Ste 100', 'crossStre...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
7,4f71cbd9e4b0d1b3a44f1b54,Copy Services,{},"{'address': '1925 San Jacinto Blvd', 'lat': 30...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
8,519aa171498e6e223908bcb1,TigerBolt Creative HQ,{},"{'lat': 30.267811, 'lng': -97.724941, 'labeled...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False
9,4ffdb412e4b0cb68a22f2a65,Napkin Venture,{},"{'address': '640 S. Congress', 'lat': 30.25545...","[{'id': '4bf58dd8d48988d1f4941735', 'name': 'D...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...",NaN,"{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636895084,[],False


In [144]:
total_venues_aus = total_venues[0:13]
len(total_venues_aus)

In [155]:
total_venues[14:28][0][1]

{'id': '4cc2a96106c25481b5428b47',
 'name': 'Uniono',
 'contact': {},
 'location': {'lat': 53.597493,
  'lng': 9.976495,
  'labeledLatLngs': [{'label': 'display', 'lat': 53.597493, 'lng': 9.976495}],
  'distance': 26,
  'postalCode': '20251',
  'cc': 'DE',
  'city': 'Hamburg',
  'state': 'Hamburg',
  'country': 'Deutschland',
  'formattedAddress': ['20251 Hamburg', 'Deutschland']},
 'categories': [{'id': '4bf58dd8d48988d1f4941735',
   'name': 'Design Studio',
   'pluralName': 'Design Studios',
   'shortName': 'Design',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/design_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'hereNow': {'count': 0, 'summary': 'Nobody here', 'groups': []},
 'referralId': 'v-1636895108',
 'venueChains': [],
 'hasPerk': False}

In [176]:
def place_store_dicc(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues_dic = {}
    aus_dic ={}
    ham_dic = {}
    ncas_dic ={}
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            if loc == "Austin":
                aus_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                print(f"{loc},{cat} hecho")
            elif loc == "Hamburg":                
                ham_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"]
                print(f"{loc},{cat} hecho")
            elif loc == "Newcastle":                
                ncas_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                print(f"{loc},{cat} hecho") 
    venues_dic = {"Austin":aus_dic,
                 "Hamburg":ham_dic,
                  "Newcastle": ncas_dic
                 }

    return venues_dic

    
    
total_venues_dic = place_store_dicc(localizations,cat_dic)

Austin,Design Studio hecho
Austin,Nursery School hecho
Austin,Preschool hecho
Austin,Elementary School hecho
Austin,Tech Startup hecho
Austin,Coffee Shop hecho
Austin,Airport Terminal hecho
Austin,Train Station hecho
Austin,Nightlife Spot hecho
Austin,Vegetarian / Vegan Restaurant hecho
Austin,Basketball Stadium hecho
Austin,Veterinarian hecho
Austin,Pet Service hecho
Austin,Pet Store hecho
Hamburg,Design Studio hecho
Hamburg,Nursery School hecho
Hamburg,Preschool hecho
Hamburg,Elementary School hecho
Hamburg,Tech Startup hecho
Hamburg,Coffee Shop hecho
Hamburg,Airport Terminal hecho
Hamburg,Train Station hecho
Hamburg,Nightlife Spot hecho
Hamburg,Vegetarian / Vegan Restaurant hecho
Hamburg,Basketball Stadium hecho
Hamburg,Veterinarian hecho
Hamburg,Pet Service hecho
Hamburg,Pet Store hecho
Newcastle,Design Studio hecho
Newcastle,Nursery School hecho
Newcastle,Preschool hecho
Newcastle,Elementary School hecho
Newcastle,Tech Startup hecho
Newcastle,Coffee Shop hecho
Newcastle,Airport Te

In [182]:
total_venues_dic

{'Austin': {'Design Studio': [{'id': '4bd339039854d13a2635fd4d',
    'name': "Kruger's Diamond Jewelers",
    'contact': {},
    'location': {'address': '722 congress Ave',
     'crossStreet': '8th',
     'lat': 30.26995307860936,
     'lng': -97.74234304510723,
     'labeledLatLngs': [{'label': 'display',
       'lat': 30.26995307860936,
       'lng': -97.74234304510723},
      {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}],
     'distance': 307,
     'postalCode': '78701',
     'cc': 'US',
     'city': 'Austin',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['722 congress Ave (8th)',
      'Austin, TX 78701',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d111951735',
      'name': 'Jewelry Store',
      'pluralName': 'Jewelry Stores',
      'shortName': 'Jewelry',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
       'suffix': '.png'},
      'primary': True}],
    'verified': True,
   

In [181]:
with open('../data/total_venues.json', 'w') as f:
    json.dump(total_venues_dic, f)

In [194]:
total_venues_dic["Austin"]
for key,value in total_venues_dic["Austin"].items():
    print(value)

[{'id': '4bd339039854d13a2635fd4d', 'name': "Kruger's Diamond Jewelers", 'contact': {}, 'location': {'address': '722 congress Ave', 'crossStreet': '8th', 'lat': 30.26995307860936, 'lng': -97.74234304510723, 'labeledLatLngs': [{'label': 'display', 'lat': 30.26995307860936, 'lng': -97.74234304510723}, {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}], 'distance': 307, 'postalCode': '78701', 'cc': 'US', 'city': 'Austin', 'state': 'TX', 'country': 'United States', 'formattedAddress': ['722 congress Ave (8th)', 'Austin, TX 78701', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d111951735', 'name': 'Jewelry Store', 'pluralName': 'Jewelry Stores', 'shortName': 'Jewelry', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_', 'suffix': '.png'}, 'primary': True}], 'verified': True, 'stats': {'tipCount': 0, 'usersCount': 0, 'checkinsCount': 0, 'visitsCount': 0}, 'beenHere': {'count': 0, 'lastCheckinExpiredAt': 0, 'marked': False, 'unconfirmedCount': 0}, 'v

In [227]:
Austin = {"Austin":[30.268735, -97.745209]}
Hamburg = {"Hamburg" : [53.5973055, 9.9762519]}
Newcastle = {"Newcastle": [54.971346, -1.611141]}
cities = {"Austin":[30.268735, -97.745209],"Hamburg" : [53.5973055, 9.9762519],"Newcastle": [54.971346, -1.611141]}
cities["Austin"][0]

30.268735

In [229]:
city = cities["Austin"]


In [233]:
def city_cat(city,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues_list = []
    

    for cat,cat_id in cat_dic.items():
        time.sleep(1)
        parametros={
            "client_id": client_id,
            "client_secret": client_secret,
            "v": "20180323",
            "ll": f"{city[0]}, {city[1]}",
            "categoryId":cat_id,
            "radius":2000
        }
        cat_dic ={cat : requests.get(url_query, params = parametros).json()["response"]["venues"]}
        venues_list.append(cat_dic)
        print(f"{cat} hecho")


    return venues_list

In [240]:
city = cities["Austin"]
#city_Austin = city_cat(city,cat_dic)
#city_Austin

In [247]:
for cat in city_Austin:
    print(cat.keys())

dict_keys(['Design Studio'])
dict_keys(['Nursery School'])
dict_keys(['Preschool'])
dict_keys(['Elementary School'])
dict_keys(['Tech Startup'])
dict_keys(['Coffee Shop'])
dict_keys(['Airport Terminal'])
dict_keys(['Train Station'])
dict_keys(['Nightlife Spot'])
dict_keys(['Vegetarian / Vegan Restaurant'])
dict_keys(['Basketball Stadium'])
dict_keys(['Veterinarian'])
dict_keys(['Pet Service'])
dict_keys(['Pet Store'])


In [249]:
def data_venues(city,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues_list = []
    

    for cat,cat_id in cat_dic.items():
        time.sleep(1)
        parametros={
            "client_id": client_id,
            "client_secret": client_secret,
            "v": "20180323",
            "ll": f"{city[0]}, {city[1]}",
            "categoryId":cat_id,
            "radius":2000
        }
        
        venues_list.append(requests.get(url_query, params = parametros).json()["response"]["venues"])
        print(f"{cat} hecho")


    return venues_list

In [250]:
city = cities["Austin"]
city_Austin = data_venues(city,cat_dic)
city_Austin

Design Studio hecho
Nursery School hecho
Preschool hecho
Elementary School hecho
Tech Startup hecho
Coffee Shop hecho
Airport Terminal hecho
Train Station hecho
Nightlife Spot hecho
Vegetarian / Vegan Restaurant hecho
Basketball Stadium hecho
Veterinarian hecho
Pet Service hecho
Pet Store hecho


[[{'id': '4bd339039854d13a2635fd4d',
   'name': "Kruger's Diamond Jewelers",
   'contact': {},
   'location': {'address': '722 congress Ave',
    'crossStreet': '8th',
    'lat': 30.26995307860936,
    'lng': -97.74234304510723,
    'labeledLatLngs': [{'label': 'display',
      'lat': 30.26995307860936,
      'lng': -97.74234304510723},
     {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}],
    'distance': 307,
    'postalCode': '78701',
    'cc': 'US',
    'city': 'Austin',
    'state': 'TX',
    'country': 'United States',
    'formattedAddress': ['722 congress Ave (8th)',
     'Austin, TX 78701',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d111951735',
     'name': 'Jewelry Store',
     'pluralName': 'Jewelry Stores',
     'shortName': 'Jewelry',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
      'suffix': '.png'},
     'primary': True}],
   'verified': True,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'chec

In [255]:
city_Austin

[[{'id': '4bd339039854d13a2635fd4d',
   'name': "Kruger's Diamond Jewelers",
   'contact': {},
   'location': {'address': '722 congress Ave',
    'crossStreet': '8th',
    'lat': 30.26995307860936,
    'lng': -97.74234304510723,
    'labeledLatLngs': [{'label': 'display',
      'lat': 30.26995307860936,
      'lng': -97.74234304510723},
     {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}],
    'distance': 307,
    'postalCode': '78701',
    'cc': 'US',
    'city': 'Austin',
    'state': 'TX',
    'country': 'United States',
    'formattedAddress': ['722 congress Ave (8th)',
     'Austin, TX 78701',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d111951735',
     'name': 'Jewelry Store',
     'pluralName': 'Jewelry Stores',
     'shortName': 'Jewelry',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
      'suffix': '.png'},
     'primary': True}],
   'verified': True,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'chec

In [256]:
Aus_venues = [venue for cat in city_Austin for venue in cat]

In [264]:
pd.DataFrame(Aus_venues)["location"].values[0]["lat"]

30.26995307860936

In [274]:
city_Austin[0]

[{'id': '4bd339039854d13a2635fd4d',
  'name': "Kruger's Diamond Jewelers",
  'contact': {},
  'location': {'address': '722 congress Ave',
   'crossStreet': '8th',
   'lat': 30.26995307860936,
   'lng': -97.74234304510723,
   'labeledLatLngs': [{'label': 'display',
     'lat': 30.26995307860936,
     'lng': -97.74234304510723},
    {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}],
   'distance': 307,
   'postalCode': '78701',
   'cc': 'US',
   'city': 'Austin',
   'state': 'TX',
   'country': 'United States',
   'formattedAddress': ['722 congress Ave (8th)',
    'Austin, TX 78701',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d111951735',
    'name': 'Jewelry Store',
    'pluralName': 'Jewelry Stores',
    'shortName': 'Jewelry',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount'

In [275]:
total

[[[{'id': '4bd339039854d13a2635fd4d',
    'name': "Kruger's Diamond Jewelers",
    'contact': {},
    'location': {'address': '722 congress Ave',
     'crossStreet': '8th',
     'lat': 30.26995307860936,
     'lng': -97.74234304510723,
     'labeledLatLngs': [{'label': 'display',
       'lat': 30.26995307860936,
       'lng': -97.74234304510723},
      {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}],
     'distance': 307,
     'postalCode': '78701',
     'cc': 'US',
     'city': 'Austin',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['722 congress Ave (8th)',
      'Austin, TX 78701',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d111951735',
      'name': 'Jewelry Store',
      'pluralName': 'Jewelry Stores',
      'shortName': 'Jewelry',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
       'suffix': '.png'},
      'primary': True}],
    'verified': True,
    'stats': {'tipCount': 0,
 

In [283]:
total_venues_dic["Austin"]["Design Studio"]

[{'id': '4bd339039854d13a2635fd4d',
  'name': "Kruger's Diamond Jewelers",
  'contact': {},
  'location': {'address': '722 congress Ave',
   'crossStreet': '8th',
   'lat': 30.26995307860936,
   'lng': -97.74234304510723,
   'labeledLatLngs': [{'label': 'display',
     'lat': 30.26995307860936,
     'lng': -97.74234304510723},
    {'label': 'entrance', 'lat': 30.269778, 'lng': -97.742379}],
   'distance': 307,
   'postalCode': '78701',
   'cc': 'US',
   'city': 'Austin',
   'state': 'TX',
   'country': 'United States',
   'formattedAddress': ['722 congress Ave (8th)',
    'Austin, TX 78701',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d111951735',
    'name': 'Jewelry Store',
    'pluralName': 'Jewelry Stores',
    'shortName': 'Jewelry',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount'

In [296]:
venues_tot = [ven for venues in total_venues for ven in venues]
All_venues = pd.DataFrame(venues_tot)

In [295]:
total_venues[0][0].keys()

dict_keys(['id', 'name', 'contact', 'location', 'categories', 'verified', 'stats', 'beenHere', 'venuePage', 'hereNow', 'referralId', 'venueChains', 'hasPerk'])

In [298]:
All_venues.to_csv("../data/all_venues",index=False)